In [33]:
# ! pip install -U langsmith
# ! pip install -qU langchain-openai
# ! pip install python-dotenv
# ! pip install -qU langchain-openai 
#! pip install "langserve[all]" 

In [35]:
import config
import os
from dotenv import load_dotenv

load_dotenv() 

os.environ["LANGCHAIN_TRACING_V2"] = "true"
# redundant
# # os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") 
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") 

In [44]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo")

In [45]:
from langchain_core.messages import HumanMessage,SystemMessage

messages = [
    SystemMessage(content="Translate tehe following from English to Italian"),
    HumanMessage(content="Hi!"),
]
model.invoke(messages)

AIMessage(content='Ciao!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 21, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8f3ecac9-cd98-435c-b543-1bf1bd7eddfe-0', usage_metadata={'input_tokens': 21, 'output_tokens': 3, 'total_tokens': 24})

In [46]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

result = model.invoke(messages)
parser.invoke(result)

'Ciao!'

In [47]:
chain = model | parser
chain.invoke(messages)

'Ciao!'

In [48]:
from langchain_core.prompts import ChatPromptTemplate
system_template = "Translate the following into {language}"
promp_template = ChatPromptTemplate.from_messages(
    [("system" , system_template),
     ("user" , "{text}")]
)
result = promp_template.invoke({"language":"italian" , "text": "hi"})

In [50]:
result.to_messages()

[SystemMessage(content='Translate teh following into italian'),
 HumanMessage(content='hi')]

In [53]:
chain = promp_template | model | parser
chain.invoke({"language":"italian" , "text": "hi"})

'Ciao'

In [52]:
chain

ChatPromptTemplate(input_variables=['language', 'text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='Translate teh following into {language}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000024850AA1B10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000024850A52E50>, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [ ]:
#refer to simpleLLm.py

# from fastapi import FastAPI
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.messages import SystemMessage, HumanMessage
# from langchain_openai import ChatOpenAI
# from langchain_core.output_parsers import StrOutputParser
# from langserve import add_routes

# system_prompt_template = "Translate the following into {language}"
# promp_template = ChatPromptTemplate.from_messages(
#     [("system", system_prompt_template),
#     ("user" , "{text}")]
# )

# model = ChatOpenAI(model = "gpt-3.5-turbo")
# parser = StrOutputParser()
# chain = promp_template | model | parser
# # res = chain.invoke({"language" : "italian",
# #               "text": "Hi"})

# app = FastAPI( title= "Langchain server" , version= "1.0" , description= " Simple API server using Langchain's Runnable Intrface")
# add_routes( app , runnable= chain , path= "/chain")
# import uvicorn
# if __name__ == "__main__":   
#     uvicorn.run(app , host= "localhost" , port = 8000)

In [ ]:
# client
from langserve import RemoteRunnable
remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian" , "text" : "hi"})